# Setup

This version of the cascaded ST model does not contain any additional components (punctuation/disfluency)

In [ ]:
!nvidia-smi

Thu Jun  9 15:59:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install bert-score transformers datasets[audio] sentencepiece sacrebleu sacremoses torchmetrics

Disfluency setup

In [ ]:
%mkdir deep_disfl_python3
%cd deep_disfl_python3

/content/deep_disfl_python3


In [ ]:
!git clone https://github.com/chuckcho/deep_disfluency.git --branch python3-port

Cloning into 'deep_disfluency'...
remote: Enumerating objects: 5146, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 5146 (delta 151), reused 136 (delta 136), pack-reused 4966
Receiving objects: 100% (5146/5146), 174.88 MiB | 17.82 MiB/s, done.
Resolving deltas: 100% (1017/1017), done.
Checking out files: 100% (2003/2003), done.


In [ ]:
!pip install sklearn-pycrfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 485 kB 6.9 MB/s 
  Created wheel for sklearn-pycrfsuite: filename=sklearn_pycrfsuite-0.4.0-py2.py3-none-any.whl size=11002 sha256=b1c1831af27f5740ff820ef378dab20e60d9a673d730e3ba028a1f4543eedab1
  Stored in directory: /root/.cache/pip/wheels/44/50/b1/295e502c6652f551dc99bfdb394a3fe2fcb47668e333391f38
  Created wheel for python-crfsuite-extension: filename=python_crfsuite_extension-0.9.7-cp37-cp37m-linux_x86_64.whl size=776005 sha256=6d0ad21cc76b9ae949d5409b201bd54b4a0ce6e20955e0c1d6b0d85814da1254
  Stored in directory: /root/.cache/pip/wheels/70/d7/bf/369f4d179407dbf3840132e4135fa61f0bf711f4496d0bf750
Successfully built sklearn-pycrfsuite python-crfsuite-extension


In [ ]:
%cd deep_disfluency
!python setup.py install

/content/deep_disfl_python3/deep_disfluency
running install
running bdist_egg
running egg_info
creating deep_disfluency.egg-info
writing deep_disfluency.egg-info/PKG-INFO
writing dependency_links to deep_disfluency.egg-info/dependency_links.txt
writing requirements to deep_disfluency.egg-info/requires.txt
writing top-level names to deep_disfluency.egg-info/top_level.txt
writing manifest file 'deep_disfluency.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'deep_disfluency.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/deep_disfluency
copying deep_disfluency/__init__.py -> build/lib/deep_disfluency
creating build/lib/deep_disfluency/language_model
copying deep_disfluency/language_model/util.py -> build/lib/deep_disfluency/language_model
copying deep_disfluency/language_model/ngram_language_model.py -> build/lib/deep_disfluency/language_model


In [ ]:
from deep_disfluency.tagger.deep_tagger import DeepDisfluencyTagger

In [ ]:
%cd /
%cp -r '/content/drive/MyDrive/Colab Notebooks/huggingface' '/root/.cache/'

/


In [ ]:
audio_folder_path = '/content/drive/MyDrive/en-de/data/train/wav/'

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForMaskedLM, Wav2Vec2Model, Wav2Vec2Processor, Wav2Vec2ForCTC, AutoModelForSeq2SeqLM
from torch import argmax
from torch import stack, cat, no_grad
import torch.optim as optim 

import pandas as pd
from datasets import Dataset
import numpy as np
from math import ceil
import torchaudio
import torch.nn.functional as F
from torchmetrics import SacreBLEUScore
# from sacrebleu.metrics import BLEU
import re
import yaml

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset

## Utility functions

In [ ]:
def remove_special_characters(s):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\—]'
    s = re.sub(chars_to_ignore_regex, '', s).lower()
    s = re.sub(r'[0-9]+', '', s)
    s = re.sub(' +', ' ', s)
    return s

def prepare_audio_batch(batch, device):
  batch_audio = []
  for i in range(len(batch['file'])):
    data = {'file': batch['file'][i], 
              'start': batch['start'][i].item(),
              'end': batch['end'][i].item(),
              'raw_text': batch['raw_text'][i],
              'text': batch['text'][i],
            }
    audio_segment = getAudioSegment(data, audio_folder_path)
    batch_audio.append(audio_segment)
  return batch_audio

def getAudioSegment(data, parent_path):
    path = parent_path + data['file']
    audio_tensor, rate = torchaudio.load(path)
    audio = audio_tensor.numpy().transpose().flatten()
    # audio = torch.FloatTensor(audio[data['start']:data['end']])
    audio = audio[data['start']:data['end']]
    del audio_tensor, path
    return audio

def read_yaml(path):
  yaml_file = None
  with open(path, "r") as stream:
      try:
          yaml_file = yaml.safe_load(stream)
      except yaml.YAMLError as exc:
          print(exc)
  return yaml_file

## Loading data

Input data

In [ ]:
path = "/content/drive/MyDrive/en-de/data/train/txt/train.yaml"
yaml_file = read_yaml(path)
en_file = open("/content/drive/MyDrive/en-de/data/train/txt/train.en", "r")
de_file = open("/content/drive/MyDrive/en-de/data/train/txt/train.de", "r")

current_wav = ""
audio, rate = None, 16000
train_data_list = []

cnt = -1
for line_props in yaml_file:
  english_text = en_file.readline()[:-2]
  deutsch_text = de_file.readline()[:-2]
  cnt += 1

  entry_dict = {}
  if line_props['wav'] != current_wav:
    current_wav = line_props['wav']
  
  punctuated_text = english_text
  reference = remove_special_characters(punctuated_text)
  if(len(reference.split(' ')) < 6):
      continue

  offset = line_props['offset']
  duration = line_props['duration']
  start = int(offset*rate)
  end = int( ceil( (offset*rate)+(duration*rate) ))

  entry_dict['file'] = current_wav
  entry_dict['start'] = start
  entry_dict['end'] = end
  entry_dict['raw_text'] = punctuated_text
  entry_dict['text'] = reference
  entry_dict['german'] = deutsch_text
  train_data_list.append(entry_dict)

In [ ]:
# df_all = pd.DataFrame(train_data_list)
# df_all.to_csv('/content/drive/MyDrive/en-de/all_chosen_data_eval_en-de.csv')
# train_data_list = None

df_all = pd.read_csv('/content/drive/MyDrive/en-de/all_chosen_data_eval_en-de.csv')
df_all.drop(axis=1, columns = [df_all.columns[0]], inplace=True)
df_all.head() 

,file,start,end,raw_text,text,german
0,ted_1.wav,98720,559520,And it's truly a great honor to have the oppor...,and it's truly a great honor to have the oppor...,"Vielen Dank, Chris. Es ist mir wirklich eine E..."
1,ted_1.wav,560160,779200,"And I say that sincerely, partly because (Mock...",and i say that sincerely partly because (mock ...,"Das meine ich ernst, teilweise deshalb — weil ..."
2,ted_1.wav,779200,1146400,Now I have to take off my shoes or boots to ge...,now i have to take off my shoes or boots to ge...,"Jetzt muss ich meine Schuhe ausziehen, um über..."
3,ted_1.wav,1161600,1227520,I'll tell you one quick story to illustrate wh...,i'll tell you one quick story to illustrate wh...,"Ich erzähle Ihnen mal eine Geschichte, dann ve..."
4,ted_1.wav,1235520,1363840,It's a true story — every bit of this is true....,it's a true story every bit of this is true so...,Eine wahre Geschichte — kein Wort daran ist er...


In [ ]:
# taking 3k sentences for evaluation
df_eval = df_all[20000:25000]

eval_dataset = Dataset.from_pandas(df_eval)
eval_dataset = eval_dataset.shuffle(seed = 42)
print(eval_dataset)

Dataset({
    features: ['file', 'start', 'end', 'raw_text', 'text', 'german'],
    num_rows: 5000
})


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(eval_dataset))

print(eval_dataset[rand_int]["raw_text"])
print(eval_dataset[rand_int]["german"])

rndm_audio = getAudioSegment(eval_dataset[rand_int], audio_folder_path)
ipd.Audio(data=np.asarray(rndm_audio), autoplay=True, rate=16000)

Just two weeks ago I made — you know, I've been making small, medium and big Klein bottles for the trade
Gerade mal vor zwei Wochen habe ich — Wissen Sie, ich habe kleine, mittlere und große Kleinflaschen für die Ausstellung gemacht


## Calculating length of training subset

In [ ]:
df_train = df_all[:15000]
df_train['length'] = (df_train.end - df_train.start)/16000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


99301.609125

In [ ]:
df_train.length.min()

0.05

In [ ]:
df_train.length.sum()/3600

27.5837803125

In [ ]:
df_eval = df_all[20000:25000]
df_eval['length'] = (df_eval.end - df_eval.start)/16000
df_eval.length.sum()/3600

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


9.69858293402778

# Model

In [ ]:
class LexicalEncoder(nn.Module):
    def __init__(self, model_path):
        super(LexicalEncoder, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForMaskedLM.from_pretrained(model_path)
    
    def forward(self, text):
        if torch.cuda.is_available():
          embedding = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to('cuda')
        else:
          embedding = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        embedding = self.model(**embedding, output_hidden_states=True)
        return embedding.hidden_states[0]
    
    def freeze_all_layers(self):
        for param in self.model.distilbert.parameters():
            param.requires_grad = False
    
class AcousticEncoder(nn.Module):
    def __init__(self, model_path) -> None:
        super(AcousticEncoder, self).__init__()

        self.processor = Wav2Vec2Processor.from_pretrained(model_path)
        self.model = Wav2Vec2Model.from_pretrained(model_path)
        self.conv_layer = nn.Conv1d(512,1024,5,stride=2)
        self.lstm_layer = nn.LSTM(input_size= 1024, hidden_size=256, dropout=0.1, bidirectional=False)
    
    def forward(self, audio):
        audio = [{"input_values": audio_seg} for audio_seg in audio]
        audio = self.processor.pad(audio, padding=True, return_tensors="pt").input_values
        if torch.cuda.is_available():
          acoustic_embedding = self.processor(audio, return_tensors="pt", padding=True, 
                                 feature_size=1, sampling_rate=16000).to('cuda')
        else:
          acoustic_embedding = self.processor(audio, return_tensors="pt", padding=True, 
                                 feature_size=1, sampling_rate=16000)
        acoustic_embedding = self.model(acoustic_embedding.input_values[0], return_dict=True).extract_features
        
        # print('then passing it to conv + LSTM layers')
        acoustic_embedding = acoustic_embedding.reshape([acoustic_embedding.shape[0], 512, acoustic_embedding.shape[-2]])
        acoustic_embedding = self.conv_layer(acoustic_embedding)

        acoustic_embedding = acoustic_embedding.reshape([acoustic_embedding.shape[0], acoustic_embedding.shape[-1], 1024])
        acoustic_embedding, _ = self.lstm_layer(acoustic_embedding)
        return acoustic_embedding

class MUSE(nn.Module):
    def __init__(self, lexical=None, acoustic=None, use_gpu=True) -> None:
        super(MUSE, self).__init__()
        
        if lexical == None:
            self.lexical_encoder = LexicalEncoder('distilbert-base-uncased').to(device)
        else:
            self.lexical_encoder = lexical

        if acoustic == None:
            self.acoustic_encoder = AcousticEncoder('facebook/wav2vec2-base-960h').to(device)
        else:
            self.acoustic_encoder = acoustic
        
        self.use_gpu = use_gpu
        self.key_projection_layer = nn.Linear(256,768)
        self.attention_layer = nn.MultiheadAttention(embed_dim = 256, num_heads = 1, dropout = 0.1)
        self.linear_layer = nn.Linear(1024, 4)
        self.softmax_layer = nn.Softmax(dim=2)
    
    def scaledDotAttention(self, query, key, value):
        key = torch.transpose(key, -1, -2)
        # print(key.shape)
        temp = torch.matmul(query,key) / np.sqrt(key.shape[-1])
        # print(temp.shape)
        attn_softmax_layer = nn.Softmax(dim = -1)
        temp = attn_softmax_layer(temp)
        # print(temp.shape)
        return torch.matmul(temp,value)

    def forward(self, text, audio):
        # print('getting lexical embedding of unpunctuated input text from pretrained lexical encoder')
        # getting lexical embedding of unpunctuated input text from pretrained lexical encoder
        lexical_embedding = self.lexical_encoder(text)

        # print('getting acoustic embedding from pretrained acoustic encoder')
        # getting acoustic embedding from pretrained acoustic encoder
        acoustic_embedding = self.acoustic_encoder(audio)

        # print('Get key, query and value from the embeddings')
        # Get key, query and value from the embeddings
        key = self.key_projection_layer(acoustic_embedding)
        query = lexical_embedding
        value = acoustic_embedding

        # print(f'Key shape {key.shape}')
        # print(f'query shape {query.shape}')
        # print(f'value shape {value.shape}')

        # print('pass key, query and value to attention')
        # pass key, query and value to multihead attention module
        attn_output = self.scaledDotAttention(query, key, value)

        # print(f'attention shape {attn_output.shape}')
        # print('concatenate attention result to lexical encoder, then pass it to linear + softmax layer')
        # concatenate attention result to lexical encoder, then pass it to linear + softmax layer
        concat_tensor = cat((attn_output, lexical_embedding), dim=2)
        # print(f'concat shape {concat_tensor.shape}')

        # freeing some memory 
        del attn_output
        del lexical_embedding

        # pass the concat tensor to the final linear + softmax layer
        output_tensor = self.linear_layer(concat_tensor)
        del concat_tensor
        # don't do softmax, it's in the crossentropy
        # output_tensor = self.softmax_layer(output_tensor)
        
        # reshape to get the following order [batch size, no. of classes, sequence len]
        output_tensor = output_tensor.reshape([output_tensor.shape[0], output_tensor.shape[-1], output_tensor.shape[-2]])

        return output_tensor

    def decode(self, logits, text_batch):
      punctuated_list = []
      
      for idx, text in enumerate(text_batch):
        id = self.lexical_encoder.tokenizer(text).input_ids
        subwords = self.lexical_encoder.tokenizer.convert_ids_to_tokens(id)
        punctuated = ""

        for i, subword in enumerate(subwords):
          if(subword == '[CLS]' or subword == '[SEP]'):
            continue
          punct_mark = None
          if(logits[idx][i] == 1):
            punct_mark = ','
          elif(logits[idx][i] == 2):
            punct_mark = '.'
          elif(logits[idx][i] == 3):
            punct_mark = '?'
          else:
            punct_mark = ''
          
          if(len(subword) <= 1):
            if(subword in ['a', 'i', '(']):
              punctuated += ' '
            punctuated += subword
          elif(len(subword) > 1 and subword[:2] == '##'):
            punctuated += subword[2:]
          else:
            if subword not in ['m', 're', 've', 's', 'd']:
              punctuated += ' ' 
            punctuated += subword
          
          punctuated += punct_mark
        
        punctuated_list.append(punctuated.strip())
      
      return punctuated_list

In [ ]:
class ASR(nn.Module):
    def __init__(self, model_path = "facebook/wav2vec2-base-960h") -> None:
        super(ASR, self).__init__()

        self.processor = Wav2Vec2Processor.from_pretrained(model_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path)

        if torch.cuda.is_available():
          self.model = self.model.cuda()
    
    def forward(self, audio):
        input_values = self.processor(audio, sampling_rate=16000, return_tensors='pt').input_values
        if torch.cuda.is_available():
          input_values = input_values.cuda()
        logits = self.model(input_values).logits
        predicted_ids = argmax(logits, dim=-1)
        transcription = self.processor.decode(predicted_ids[0])

        del input_values, logits, predicted_ids
        return transcription
    
class Translator(nn.Module):
    def __init__(self, model_path = "facebook/wmt19-en-de"):
        super(Translator, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

        if torch.cuda.is_available():
          self.model = self.model.cuda()

    def forward(self, text):
        input_ids = self.tokenizer(text, return_tensors = 'pt').input_ids
        if torch.cuda.is_available():
          input_ids = input_ids.cuda()
        outputs = self.model.generate(input_ids)
        translation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        del input_ids, outputs

        return translation

class DisfluencyModel:
    def __init__(self):
        self.tagger = DeepDisfluencyTagger(
            config_file="/content/deep_disfl_python3/deep_disfluency/deep_disfluency/experiments/experiment_configs.csv",
            config_number=21,
            saved_model_dir="/content/deep_disfl_python3/deep_disfluency/deep_disfluency/experiments/021/epoch_40"
            )
    
    def tag_disfluency(self, text):
        words = text.split()
        for word in words:
            self.tagger.tag_new_word(word)
        # print('tagging finished...')
        return self.remove_disfluency(words)
        
    
    def remove_disfluency(self, words):
        new_text = ''
        for w, t in zip(words, self.tagger.output_tags):
            # print(w, '\t', t)
            if t == '<f/>' or t[:4] == '<rps':
                new_text += w + ' '
        
        self.reset()
        return new_text.strip()
    
    def reset(self):
        self.tagger.reset()

class CascadedST(nn.Module):
    def __init__(self, with_punct = False, with_disf = False):
        super(CascadedST, self).__init__()

        if torch.cuda.is_available():
          self.asr = ASR().cuda()
        
        self.with_punct = with_punct
        if with_punct:
            self.muse_model = MUSE()
            self.muse_model.load_state_dict(torch.load('/content/drive/MyDrive/MuSe model/saved_model_batch4.pth'))
            self.muse_model = self.muse_model.cuda()
            self.muse_model.acoustic_encoder.model.freeze_feature_encoder()

        self.with_disf = with_disf
        if with_disf:
            self.disf_model = DisfluencyModel()

        if torch.cuda.is_available():
          self.translator = Translator().cuda()
        
        if torch.cuda.is_available():
          self.asr = self.asr.cuda()
          self.translator = self.translator.cuda()
        

    def forward(self, audio):
        if(self.with_punct == True):
            transcript = self.asr(audio[0])
        else:
            transcript = self.asr(audio)

        if self.with_disf:
            transcript = self.disf_model.tag_disfluency(transcript)
        
        if self.with_punct:
            pred = self.muse_model(transcript, audio)
            logits = torch.argmax(pred, dim=-2)
            transcript = self.muse_model.decode(logits, [transcript])
        
        translation = self.translator(transcript)
        
        return translation

#Trying things first

In [ ]:
dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=1)
batch = next(iter(dataloader))

In [ ]:
audio = torch.Tensor(prepare_audio_batch(batch, device)).cuda()

In [ ]:
asr = ASR().to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
transcript = asr(audio[0])
transcript

'WOMEN SIT AT TABLES AND MEN GO FROM TABLE TO TABLE NEARS A CLOCK AND A BELL IN AT FIVE MINUTES MINGO THE CONVERSATION ENDS AND THE WOMAN CAN DECIDE WHETHER TO GIVE HER CARD OR HER EMALE ADDRESSED THE MAN FOR FALAP'

In [ ]:
disf_detector = DisfluencyModel()
transcript = disf_detector.tag_disfluency(transcript)
transcript

Initializing Tagger
Processing args from config number 21 ...
Intializing model from args...
Using the cpu
	Adjust Theano config file ($HOME/.theanorc)
loading tag to index maps...
Initializing model of type elman ...


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Loading saved weights from /content/deep_disfl_python3/deep_disfluency/deep_disfluency/experiments/021/epoch_40
No POS tagger specified,loading default CRF switchboard one
Not using timing data
Loading decoder...
loading swbd_disf1_021 Markov model
No timing model given
Markov Model ready mode:
constraint only


'WOMEN SIT AT TABLES AND MEN GO FROM TABLE TO TABLE NEARS A CLOCK AND A BELL AT FIVE MINUTES MINGO THE CONVERSATION ENDS AND THE WOMAN CAN DECIDE WHETHER TO GIVE HER CARD OR HER EMALE ADDRESSED THE MAN FOR FALAP'

In [ ]:
punct_model = MUSE().cuda()

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout afte

In [ ]:
logits = torch.argmax(punct_model(transcript, audio), dim=-2)
print(logits.shape)
transcript = punct_model.decode(logits, [transcript])
transcript

torch.Size([1, 49])


['women? sit. at? tables and, men. go, from, table, to. table. near?s? a clock, and a. bell, at five minutes? ming?o the conversation? ends. and. the, woman? can? decide, whether, to? give her, card? or. her. emale? addressed the, man. for? fa?la?p.']

In [ ]:
translator = Translator().to(device)

In [ ]:
translation = translator(transcript)

In [ ]:
print(translation)
print(batch['german'])

Frauen? sitzen an? Tischen und, men. gehen, von, Tisch, zu. Tafel. in der Nähe? s? eine Uhr, und a. Glocke, um fünf Minuten? ming? o das Gespräch? endet. and. die, Frau? kann? entscheiden, ob sie? ihr, Karte? geben soll oder her. emale? adressierte die, man. für? fa? la? p.
['Frauen sitzen an Tischen und Männer gehen von Tisch zu Tisch, und es gibt eine Uhr und eine Glocke und nach fünf Minuten endet das Gespräch und die Frau kann entscheiden, ob sie ihm ihre Karte oder E-Mail-Adresse gibt, zum weiteren Kontakt']


In [ ]:
# bleu = BLEU(lowercase=True)
# score = bleu.corpus_score([translation], [batch['german']])
# score

BLEU = 20.77 53.1/27.1/14.9/8.7 (BP = 1.000 ratio = 1.089 hyp_len = 49 ref_len = 45)

In [ ]:
from torchmetrics import SacreBLEUScore
metric = SacreBLEUScore(smooth=True, lowercase = True)
metric([translation], [batch['german']])

tensor(0.2483)

# Evaluating model variants

In [ ]:
def eval_model(model, dataset, device):
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=1)
  translations = []
  # references = []
  for idx, batch in enumerate(dataloader):
    if idx%500==0:
      print(f'current batch = {idx}')
    try:
    #   audio = prepare_audio_batch(batch, device)[0]
      audio = torch.Tensor(prepare_audio_batch(batch, device)).cuda()
      translated_text = model(audio)
      translations.append(translated_text)
      # references.append(batch['german'][0])
    except Exception as ex:
      print(idx, ex)
    #   raise ex
    # break
  
  metric = SacreBLEUScore(smooth=True, lowercase = True)
  score = metric(translations,[dataset['german']])

  return translations, score

## Vanilla variant

In [ ]:
vanilla_model = CascadedST().to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
translations, vanilla_score = eval_model(vanilla_model, eval_dataset, device)
print(vanilla_score)

current batch = 0
current batch = 500
current batch = 1000
current batch = 1500
current batch = 2000
current batch = 2500
current batch = 3000
current batch = 3500
current batch = 4000
current batch = 4500
BLEU = 6.74 34.0/10.7/4.3/1.9 (BP = 0.919 ratio = 0.922 hyp_len = 96684 ref_len = 104859)


BLEU = 6.74 34.0/10.7/4.3/1.9 (BP = 0.919 ratio = 0.922 hyp_len = 96684 ref_len = 104859)

In [ ]:
metric = SacreBLEUScore(smooth=True, lowercase = True)
metric(translations,[eval_dataset['german']])

tensor(0.4329)

With smoothing and lowercasing: 0.4329

## With Disfluency Only

In [ ]:
cascaded_model_disf = CascadedST(with_disf=True).to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initializing Tagger
Processing args from config number 21 ...
Intializing model from args...
Using the cpu
	Adjust Theano config file ($HOME/.theanorc)
loading tag to index maps...
Initializing model of type elman ...


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Loading saved weights from /content/deep_disfl_python3/deep_disfluency/deep_disfluency/experiments/021/epoch_40
No POS tagger specified,loading default CRF switchboard one
Not using timing data
Loading decoder...
loading swbd_disf1_021 Markov model
No timing model given
Markov Model ready mode:
constraint only


In [ ]:
import gc
gc.collect()

206

In [ ]:
translations_disf, score_with_disf = eval_model(cascaded_model_disf, eval_dataset, device)
print(score_with_disf)

current batch = 0
current batch = 500
current batch = 1000
current batch = 1500
current batch = 2000
current batch = 2500
current batch = 3000
current batch = 3500
current batch = 4000
current batch = 4500
tensor(0.4265)


## With punctuation prediction only

In [ ]:
cascaded_model_punct = CascadedST(with_punct=True).to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and

In [ ]:
translations_punct, score_with_punct = eval_model(cascaded_model_punct, eval_dataset, device)
print(score_with_punct)

current batch = 0
current batch = 500
current batch = 1000
current batch = 1500
current batch = 2000
current batch = 2500
2714 Calculated padded input size per channel: (2). Kernel size: (5). Kernel size can't be greater than actual input size
current batch = 3000
current batch = 3500
current batch = 4000
current batch = 4500
tensor(0.4972)


## Disfluency with Punctuation Prediction

In [ ]:
cascaded_model_disf_punct = CascadedST(with_punct=True, with_disf=True).to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and

Initializing Tagger
Processing args from config number 21 ...
Intializing model from args...
Using the cpu
	Adjust Theano config file ($HOME/.theanorc)
loading tag to index maps...
Initializing model of type elman ...
Loading saved weights from /content/deep_disfl_python3/deep_disfluency/deep_disfluency/experiments/021/epoch_40
No POS tagger specified,loading default CRF switchboard one
Not using timing data
Loading decoder...
loading swbd_disf1_021 Markov model
No timing model given
Markov Model ready mode:
constraint only


In [ ]:
translations_disf_punct, score_with_disf_punct = eval_model(cascaded_model_disf_punct, eval_dataset, device)
print(score_with_disf_punct)

current batch = 0
current batch = 500
640 CUDA out of memory. Tried to allocate 348.00 MiB (GPU 0; 14.76 GiB total capacity; 12.85 GiB already allocated; 231.75 MiB free; 13.13 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
current batch = 1000
current batch = 1500
current batch = 2000
current batch = 2500
2714 Calculated padded input size per channel: (2). Kernel size: (5). Kernel size can't be greater than actual input size
current batch = 3000
current batch = 3500
current batch = 4000
current batch = 4500
tensor(0.4981)
